# Create a Simple Reflex-Based Lunar Lander Agent

In this example, we will use Gymnasium, an environment to train agents via reinforcement learning (RL). We will not use RL here but just use the environment with a custom simple reflex-based agent. 

## Install Gymnasium

The documentation for Gymnasium is available at https://gymnasium.farama.org/ 

Steps:
1. Create a new folder and open it with VS Code and install all needed Python Extensions in VS Code.
2. Create a new virtual environment (CTRL-Shift P Python Create Environment...)
3. I needed to install swig and the Python C++ headers on WSL2 via the terminal
    * `sudo apt install swig`
    * `sudo apt-get install python3-dev` 
4. Install gymnasium with the needed extras

In [1]:
%pip install -q swig
%pip install -q gymnasium[box2d,classic_control]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## The Lunar Lander Environment 

The documentation of the environment is available at: https://gymnasium.farama.org/environments/box2d/lunar_lander/

* Performance Measure: A reward of -100 or +100 points for crashing or landing safely respectively. We do not use 
  intermediate rewards here.

* Environment: This environment is a classic rocket trajectory optimization problem. A ship needs to land safely. The space is **continuous** with
  x and y coordinates in the range [-2.5, 2.5]. The landing pad is at coordinate (0,0).

* Actuators:  According to Pontryagin’s
  maximum principle, it is optimal to fire the engine at full throttle or turn it off. This is the reason why this environment has discrete actions: engine on or off. There are four discrete actions available:

    - 0: do nothing
    - 1: fire left orientation engine
    - 2: fire main engine
    - 3: fire right orientation engine

* Sensors: Each observation is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

Gymnasim environments are implemented as classes with a `make` method to create the environment, a `reset` method, and a `step` method to execute an action.
To use it with an agent function that expects percetps and returns an action, we need write glue code that connects the environment with the agent function.

In [2]:
import gymnasium as gym

def run_episode(agent_function, max_steps=1000):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialize the environment
    env = gym.make("LunarLander-v3", render_mode="human")

    # Reset the environment to generate the first observation (use seed=42 in reset to get reproducible results)
    observation, info = env.reset()

    # run one episode
    for _ in range(max_steps):
        # call the agent function to select an action
        action = agent_function(observation)

        print (f"Obs: {observation} -> Action: {action}")

        # step: execute an action in the environment
        observation, reward, terminated, truncated, info = env.step(action)
        
        env.render()

        if terminated:
            print(f"Final Reward: {reward}")
            break
    
    env.close()
    return reward

Note: `env.render()` shows the environment when the notebook is locally run (e.g., in VScode). On Colab, you cannot see the environment because the code is run on a headless server (i.e., a server without a display). There are some workarounds you can google.

## Example: A Random Agent

We ranomly return one of the actions. The environment accepts the integers 0-3.


In [28]:
import numpy as np

def random_agent_function(observation): 
    """A random agent that selects actions uniformly at random. It ignores the observation."""
    return np.random.choice([0, 1, 2, 3], p=[0.25, 0.25, 0.25, 0.25])

run_episode(random_agent_function)

Obs: [-0.00165844  1.4052174  -0.16799843 -0.2534484   0.00192851  0.03805415
  0.          0.        ] -> Action: 0
Obs: [-0.00331707  1.3989378  -0.16777131 -0.27909806  0.00380884  0.03761082
  0.          0.        ] -> Action: 3
Obs: [-0.004916    1.3920658  -0.16026768 -0.30543184  0.00418165  0.0074571
  0.          0.        ] -> Action: 1
Obs: [-0.00661316  1.3845807  -0.17258437 -0.33267027  0.00702575  0.0568872
  0.          0.        ] -> Action: 3
Obs: [-0.00821209  1.3764875  -0.16027372 -0.3597073   0.00739871  0.00745986
  0.          0.        ] -> Action: 3
Obs: [-0.00972767  1.3677849  -0.14981604 -0.38676944  0.00567749 -0.03442758
  0.          0.        ] -> Action: 1
Obs: [-0.01133223  1.3584768  -0.16099149 -0.41369528  0.00619915  0.01043418
  0.          0.        ] -> Action: 2
Obs: [-1.3041782e-02  1.3498448e+00 -1.7099206e-01 -3.8364503e-01
  6.2450748e-03  9.1847137e-04  0.0000000e+00  0.0000000e+00] -> Action: 3
Obs: [-0.01466427  1.3406031  -0.16008726 

-100

## A Simple Reflex-Based Agent

To make the code easier to read, we use enumerations for actions (integers) and observations (index in the observation vector).

---

```python

observation = [
    x,                # vị trí ngang
    y,                # vị trí dọc
    vx,               # vận tốc ngang
    vy,               # vận tốc dọc
    angle,            # góc nghiêng
    angular_velocity, # vận tốc quay
    left_leg_contact, # 1 nếu chân trái chạm đất
    right_leg_contact # 1 nếu chân phải chạm đất
]

```
---

In [4]:
from enum import Enum

class Act(Enum):
    LEFT = 1
    RIGHT = 3
    MAIN = 2
    NO_OP = 0

class Obs(Enum):
    X = 0
    Y = 1
    VX = 2
    VY = 3
    ANGLE = 4
    ANGULAR_VELOCITY = 5
    LEFT_LEG_CONTACT = 6
    RIGHT_LEG_CONTACT = 7


In [25]:
def rocket_agent_function(observation):
    """A simple agent function."""

    # run the main thruster, if the lander is falling too fast
    if observation[Obs.VY.value] < -.3:  
        return Act.MAIN.value

    return Act.NO_OP.value 

run_episode(rocket_agent_function)

Obs: [ 0.00329008  1.4041309   0.33323437 -0.30173886 -0.0038056  -0.07548247
  0.          0.        ] -> Action: 2
Obs: [ 0.0065814   1.3982518   0.33292055 -0.26132205 -0.00754735 -0.07484166
  0.          0.        ] -> Action: 0
Obs: [ 0.00987291  1.3917717   0.33293232 -0.28802222 -0.01128672 -0.07479437
  0.          0.        ] -> Action: 0
Obs: [ 0.01316462  1.3846921   0.33294338 -0.3146895  -0.01502565 -0.0747852
  0.          0.        ] -> Action: 2
Obs: [ 0.01630745  1.3785666   0.31886292 -0.27229518 -0.01955076 -0.09051023
  0.          0.        ] -> Action: 0
Obs: [ 0.01945057  1.3718411   0.3188761  -0.29897764 -0.02407445 -0.09048234
  0.          0.        ] -> Action: 0
Obs: [ 0.02259379  1.364516    0.31888935 -0.32564184 -0.0285978  -0.09047497
  0.          0.        ] -> Action: 2
Obs: [ 0.02568474  1.3579974   0.31405577 -0.28981283 -0.03349668 -0.09798674
  0.          0.        ] -> Action: 0
Obs: [ 0.02877598  1.3508791   0.3140697  -0.31649458 -0.03839445

-100

## Evaluating the Agent

Run the agent on 100 problems and report the average reward.

In [6]:
import numpy as np

def run_episode_test(agent_function):
    """Run one episode in the LunarLander-v3 environment using the provided agent."""

    # Initialise the environment
    env = gym.make("LunarLander-v3", render_mode=None)

    # Reset the environment to generate the first observation
    observation, info = env.reset()

    # run one episode (max. 1000 steps)
    for _ in range(1000):
        # call the agent to select an action
        action = agent_function(observation)

        # step (transition) through the environment with the action
        observation, reward, terminated, truncated, info = env.step(action)
      
        if terminated:
            break
    
    env.close()
    return reward

def run_episodes(agent_function, n=1000):
    """Run multiple episodes with the given agent and return the rewards for each episode."""
    return [run_episode_test(agent_function) for _ in range(n)]

rewards = run_episodes(rocket_agent_function)
print(rewards)

print(f"Average reward: {np.average(rewards)}")
print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 100, -100, 100, -100, -100, -100, -100, -100, -100, -100,

This is not great!


## Implement A Better Reflex-Based Agent

Build a better that uses its right and left thrusters to land the craft (more) safely. Test your agent function using 100 problems.


**Cách tính reward trong LunarLander**

- Điểm số (reward) không chỉ dựa vào vị trí X khi hạ cánh.

- Môi trường chấm điểm theo nhiều yếu tố:

1. Tiến gần vùng hạ cánh (giữa hai cột cờ) → thưởng dương.

2. Sử dụng động cơ chính (main thruster) → phạt nhỏ mỗi lần (để khuyến khích tiết kiệm nhiên liệu).

3. Động cơ bên (left/right thrusters) → cũng bị phạt nhỏ.

4. Rơi ra ngoài màn hình hoặc crash mạnh xuống đất → phạt -100 hoặc hơn.

5. Hạ cánh thành công, ổn định (giữa cột cờ, tốc độ thấp, gần như đứng yên, hai chân chạm đất) → thưởng +100 … +140 điểm.


**Code rơi đúng vào chỗ 2 cột cờ nhưng sẽ bị tai nạn vì rơi quá nhanh**

In [24]:
def better_rocket_agent_function(observation):
    X = observation[Obs.X.value]    # vị trí ngang
    Y = observation[Obs.Y.value]    # vị trí dọc 
    VX = observation[Obs.VX.value]  # vận tốc ngang
    VY = observation[Obs.VY.value]  # vận tốc dọc
    ANGLE = observation[Obs.ANGLE.value]    # góc nghiêng
    ANGULAR_VELOCITY = observation[Obs.ANGULAR_VELOCITY.value]  # vận tốc quay 
    LEFT_LEG_CONTACT = observation[Obs.LEFT_LEG_CONTACT.value]  # nếu chân trái chạm đất  
    RIGHT_LEG_CONTACT = observation[Obs.RIGHT_LEG_CONTACT.value]    # nếu chân phải chạm đất
    
    # Decision Threshold
    ANGLE_THRESHOLD = 0.2
    VX_THRESHOLD = 0.005
    VY_THRESHOLD = 0.2
    X_THRESHOLD = 0.005
    
    if X == 0 and Y == 0 :
        return Act.NO_OP.value
    # 1. Điều chỉnh khi bị nghiêng
    if ANGLE > ANGLE_THRESHOLD:
        return Act.RIGHT.value
    elif ANGLE < -ANGLE_THRESHOLD:
        return Act.LEFT.value

    # 2. Điều chỉnh vị trí ngang về 0
    elif abs(X) > X_THRESHOLD:
        if X > 0:  
            return Act.RIGHT.value
        else: 
            return Act.LEFT.value
    
    # 3. Kiểm soát vận tốc
    elif abs(VX) > VX_THRESHOLD and Y < 1.0:
        if VX > 0:
            return Act.RIGHT.value 
        else:
            return Act.LEFT.value
    
    # 4. Kiểm soát vận tốc rơi
    elif VY <  - VY_THRESHOLD:
        return Act.MAIN.value
    
    # 5. Không làm gì cả
    else:
        return Act.NO_OP.value
    
        
    
print("=== Test ===")
rewards = run_episode(better_rocket_agent_function , 150)
rewards

# rewards = run_episodes(better_rocket_agent_function)
# print(rewards)

# print(f"Average reward: {np.average(rewards)}")
# print(f"Success rate: {np.sum(np.array(rewards) == 100)}/{len(rewards)}")

=== Test ===
Obs: [-0.00605555  1.4099116  -0.6133875  -0.04483952  0.00702376  0.13894156
  0.          0.        ] -> Action: 1
Obs: [-0.01220722  1.4083136  -0.6245433  -0.07110202  0.01629468  0.18543616
  0.          0.        ] -> Action: 1
Obs: [-0.0184556   1.4061257  -0.6366519  -0.09741064  0.02797999  0.23372786
  0.          0.        ] -> Action: 1
Obs: [-0.02477245  1.4033476  -0.64521873 -0.12377953  0.0413698   0.26782116
  0.          0.        ] -> Action: 1
Obs: [-0.03115921  1.3999795  -0.65396625 -0.150187    0.0564977   0.30258614
  0.          0.        ] -> Action: 1
Obs: [-0.03761892  1.3959984  -0.66310006 -0.17767791  0.07345965  0.33927068
  0.          0.        ] -> Action: 1
Obs: [-0.04417152  1.3914092  -0.6747077  -0.2050345   0.09274837  0.38581026
  0.          0.        ] -> Action: 1
Obs: [-0.0507946   1.3862243  -0.6834812  -0.23189446  0.11377918  0.42065492
  0.          0.        ] -> Action: 1
Obs: [-0.05751209  1.380428   -0.6952294  -0.259570

-100